In [189]:
import requests
import re
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By

In [60]:
# driver = webdriver.Chrome()

## UCI

In [81]:
uci_base = "https://catalogue.uci.edu"

In [82]:
uci_all = requests.get(uci_base + '/undergraduatedegrees/').text
uci_soup = BeautifulSoup(uci_all, 'html.parser')

In [112]:
uci_programs = uci_soup.select_one('h2[class="tglhead"]').find_next_siblings()[0].select('li')

In [120]:
uci_names = []
uci_reqs = []
for p in tqdm(uci_programs):
    name = p.text.split(',')
    if 'B' not in name[-1]:
        pass
    else:
        uci_names.append(''.join(name[:-1]))
        p_url = uci_base + p.a['href'] + "#requirementstext"
        p_reqs = requests.get(p_url).text
        p_soup = BeautifulSoup(p_reqs, 'html.parser')
        p_reqs_ls = list(map(lambda x: x.text.strip(), p_soup.select('td[class="codecol"] + td:not([class]):not([colspan])')))
        uci_reqs.append(', '.join(p_reqs_ls))

100%|██████████| 166/166 [00:27<00:00,  6.08it/s]


In [124]:
uci_df = pd.DataFrame({'program': uci_names, 'requirements': uci_reqs})

In [125]:
uci_df.head()

,program,requirements
0,Aerospace Engineering,"General Chemistry, Accelerated General Chemist..."
1,African American Studies,"African American Studies I, African American S..."
2,Anthropology,"Introduction to Sociocultural Anthropology, In..."
3,Applied Physics,"Single-Variable Calculus, Single-Variable Calc..."
4,Art,"Art in Context: History, Theory, and Practice,..."


## Princeton

In [170]:
pu_base = "https://ua.princeton.edu"

In [171]:
pu_ba = requests.get(pu_base + '/academic-units/departmental-concentrations-for-the-degree-of-bachelor-of-arts').text
pu_ba_soup = BeautifulSoup(pu_ba, 'html.parser')
pu_bse = requests.get(pu_base + '/academic-units/departmental-concentrations-for-the-degree-of-bachelor-of-science-in-engineering').text
pu_bse_soup = BeautifulSoup(pu_bse, 'html.parser')

In [172]:
pu_ba_programs = pu_ba_soup.select(".field-content > a")
pu_bse_programs = pu_bse_soup.select(".field-content > a")

In [217]:
pu_names = []
pu_courses = []
pu_desc = []
for p in tqdm(pu_ba_programs + pu_bse_programs):
    name = p.text.strip()
    pu_names.append(name)
    p_url = pu_base + pu_ba_programs[0]['href']
    p_content = requests.get(p_url).text
    p_soup = BeautifulSoup(p_content, 'html.parser')
    p_courses = ' '.join(list(map(lambda x: re.sub('\n', '', ' '.join(x.text.strip().split(' ')[2:])), p_soup.select('h3[class="course-title"]'))))
    pu_courses.append(p_courses)
    p_desc = ' '.join(list(map(lambda x: x.text.strip(), p_soup.select('.course-desc'))))
    pu_desc.append(p_desc)

100%|██████████| 37/37 [00:09<00:00,  3.86it/s]


In [220]:
pu_df = pd.DataFrame({'institution': 'Princeton', 'program': pu_names, 'courses': pu_courses, 'descriptions': pu_desc})

In [221]:
pu_df.head()

,institution,program,courses,descriptions
0,Princeton,African American Studies,African American Studies and the Philosophy of...,This course introduces students to the field o...
1,Princeton,Anthropology,African American Studies and the Philosophy of...,This course introduces students to the field o...
2,Princeton,Architecture,African American Studies and the Philosophy of...,This course introduces students to the field o...
3,Princeton,Art and Archaeology,African American Studies and the Philosophy of...,This course introduces students to the field o...
4,Princeton,Astrophysical Sciences,African American Studies and the Philosophy of...,This course introduces students to the field o...


In [ ]:
pu_df.to_csv('princeton_programs.csv', index=False)